# Getting Started with Tools and Agents in Mirascope

This notebook provides a detailed introduction to using Tools and implementing Agents in Mirascope. We'll use the WebSearchAgent as our primary example to demonstrate these concepts.

1. [Introduction](#Introduction)
2. [Setting Up the Environment](#Setting-Up-the-Environment)
3. [Building a Basic Chatbot](#Building-a-Basic-Chatbot)
4. [Understanding Tools in Mirascope](#Understanding-Tools-in-Mirascope)
5. [Creating Custom Tools](#Creating-Custom-Tools)
6. [Implementing the WebSearchAgent](#Implementing-the-WebSearchAgent)
7. [Running and Testing the Agent](#Running-and-Testing-the-Agent)
8. [Advanced Concepts and Best Practices](#Advanced-Concepts-and-Best-Practices)
9. [Conclusion](#Conclusion)

## 1. Introduction

Tools and Agents are two key concepts in building advanced AI systems, particularly those involving Large Language Models (LLMs):

- **Tools**: Functions that extend the capabilities of LLMs, allowing them to perform specific tasks or access external data.
- **Agents**: Autonomous or semi-autonomous entities that use LLMs and Tools to perform complex tasks or interact with users.

In this notebook, we'll explore how to create and use Tools, and how to implement an Agent using the WebSearchAgent as our example. We'll be using Mirascope, a toolkit that simplifies the process of building LLM-powered applications with tools and agents.

For more detailed information on these concepts, refer to the following Mirascope documentation:

- [Tools documentation](https://docs.mirascope.io/latest/learn/tools/)
- [Agents documentation](https://docs.mirascope.io/latest/learn/agents/)

## 2. Setting Up the Environment

First, let's set up our environment by installing Mirascope and other necessary libraries.

In [ ]:
!pip install "mirascope[openai]" duckduckgo_search beautifulsoup4 requests

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your-api-key-here"

For more information on setting up Mirascope and its dependencies, see the [Mirascope installation guide](https://docs.mirascope.io/latest/get-started/).

## 3. Building a Basic Chatbot

Before we dive into Tools and Agents, let's start by building a basic chatbot. This will help us understand the fundamental concepts of state management and conversation flow in Mirascope.

In [1]:
from pydantic import BaseModel

from mirascope.core import BaseMessageParam, openai, prompt_template


class BasicChatbot(BaseModel):
    messages: list[BaseMessageParam | openai.OpenAIMessageParam] = []

    @openai.call(model="gpt-4o-mini")
    @prompt_template(
        """
        SYSTEM: You are a friendly chatbot assistant. Engage in a conversation with the user.
        MESSAGES: {self.messages}
        USER: {user_input}
        """
    )
    async def chat(self, user_input: str): ...

    async def run(self):
        while True:
            user_input = input("User: ")
            if user_input.lower() == "exit":
                break
            response = await self.chat(user_input)
            print(f"User: {user_input}")
            print(f"Chatbot: {response.content}")
            self.messages.append(response.user_message_param)
            self.messages.append(response.message_param)


# Usage
chatbot = BasicChatbot()
# Run the chatbot in a Jupyter notebook
await chatbot.run()

# Run the chatbot in a Python script
# import asyncio
# asyncio.run(chatbot.run())

User: Good morning!
Chatbot: Good morning! How are you today?
User: I'm doing well, thank you! How about you?
Chatbot: I’m doing great, thanks for asking! Is there anything specific you’d like to chat about today?


In this basic chatbot implementation, we've created a `BasicChatbot` class that maintains a conversation history (`messages`) and uses Mirascope's `@openai.call` and `@prompt_template` decorators to interact with the LLM. Note that the `chat` method can directly access `self.messages` in the template, allowing for easy integration of the conversation history into the prompt.

The `chat` method is where the LLM interaction occurs. It uses the conversation history and the current user input to generate a response. The `run` method manages the main conversation loop, updating the conversation history after each interaction.

This sets the foundation for more complex agents that we'll build upon in the following sections.

## 4. Understanding Tools in Mirascope

Tools in Mirascope are functions that extend the capabilities of LLMs. They allow the LLM to perform specific tasks or access external data. Tools are typically used to:

1. Retrieve information from external sources
2. Perform calculations or data processing
3. Interact with APIs or databases
4. Execute specific actions based on the LLM's decisions

Let's start by creating a Tool that extracts content from a webpage:

In [2]:
import re

import requests
from bs4 import BeautifulSoup


def extract_content(url: str) -> str:
    """Extract the main content from a webpage.

    Args:
        url: The URL of the webpage to extract the content from.

    Returns:
        The extracted content as a string.
    """
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.content, "html.parser")
        unwanted_tags = ["script", "style", "nav", "header", "footer", "aside"]
        for tag in unwanted_tags:
            for element in soup.find_all(tag):
                element.decompose()
        main_content = (
            soup.find("main")
            or soup.find("article")
            or soup.find("div", class_=re.compile("content|main"))
        )
        if main_content:
            text = main_content.get_text(separator="\n", strip=True)
        else:
            text = soup.get_text(separator="\n", strip=True)
        lines = (line.strip() for line in text.splitlines())
        return "\n".join(line for line in lines if line)
    except Exception as e:
        return f"{type(e)}: Failed to extract content from URL {url}"

This `extract_content` function is a Tool that takes a URL as input and returns the main content of the webpage as a string. It uses [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) to parse the HTML and extract the relevant text content.

When this function is passed to a Mirascope `call` decorator, it is automatically converted into a `BaseTool` class. This conversion process generates a schema that the LLM API uses to understand and interact with the tool.

Let's take a look at what this schema looks like:

In [3]:
from mirascope.core.openai import OpenAITool

tool_type = OpenAITool.type_from_fn(extract_content)
print(tool_type.tool_schema())

{'function': {'name': 'extract_content', 'description': 'Extract the main content from a webpage.\n\nArgs:\n    url: The URL of the webpage to extract the content from.\n\nReturns:\n    The extracted content as a string.', 'parameters': {'properties': {'url': {'description': 'The URL of the webpage to extract the content from.', 'type': 'string'}}, 'required': ['url'], 'type': 'object'}}, 'type': 'function'}


This schema provides the LLM with information about the tool's name, description, and expected input parameters. The LLM uses this information to determine when and how to use the tool during its reasoning process.

For more details on implementing and using Tools in Mirascope, see the [Tools documentation](https://docs.mirascope.io/latest/learn/tools/).

## 5. Tools With Access To Agent State

Now, let's create a more complex Tool that performs web searches using the DuckDuckGo search engine. We'll implement this tool as a method within our agent class to demonstrate how tools can access the agent's state:

In [4]:
from duckduckgo_search import DDGS


class WebSearchAgentBase(BaseModel):
    messages: list[BaseMessageParam | openai.OpenAIMessageParam] = []
    search_history: list[str] = []
    max_results_per_query: int = 2

    def web_search(self, queries: list[str]) -> str:
        """Performs web searches for given queries and returns URLs.

        Args:
            queries: List of search queries.

        Returns:
            str: Newline-separated URLs from search results or error messages.

        Raises:
            Exception: If web search fails entirely.
        """
        try:
            urls = []
            for query in queries:
                results = DDGS(proxies=None).text(
                    query, max_results=self.max_results_per_query
                )
                for result in results:
                    link = result["href"]
                    try:
                        urls.append(link)
                    except Exception as e:
                        urls.append(
                            f"{type(e)}: Failed to parse content from URL {link}"
                        )
            self.search_history.extend(queries)
            return "\n\n".join(urls)
        except Exception as e:
            return f"{type(e)}: Failed to search the web for text"

This `web_search` method is a more complex Tool that takes a list of search queries and returns a string of newline-separated URLs from the search results. It uses the DuckDuckGo search API to perform the searches and updates the agent's `search_history`. Luckily, the DuckDuckGo search API does not require an API key, making it easy to use in this example.

By implementing this tool as a method within our `WebSearchAgent` class, we can access and update the agent's state (like `search_history`) directly. This approach allows for more integrated and stateful tool usage within our agent.

## 6. Implementing the WebSearchAgent

Now that we have our custom tools, let's implement the full WebSearchAgent by adding the LLM interaction and main execution flow:

In [5]:
from datetime import datetime


class WebSearchAgent(WebSearchAgentBase):
    @openai.call(model="gpt-4o-mini", stream=True)
    @prompt_template(
        """
        SYSTEM:
        You are an expert web searcher. Your task is to answer the user's question using the provided tools.
        The current date is {current_date}.

        You have access to the following tools:
        - `web_search`: Search the web when the user asks a question. Follow these steps for EVERY web search query:
            1. There is a previous search context: {self.search_history}
            2. There is the current user query: {question}
            3. Given the previous search context, generate multiple search queries that explores whether the new query might be related to or connected with the context of the current user query. 
                Even if the connection isn't immediately clear, consider how they might be related.
        - `extract_content`: Parse the content of a webpage.

        When calling the `web_search` tool, the `body` is simply the body of the search
        result. You MUST then call the `extract_content` tool to get the actual content
        of the webpage. It is up to you to determine which search results to parse.

        Once you have gathered all of the information you need, generate a writeup that
        strikes the right balance between brevity and completeness based on the context of the user's query.

        MESSAGES: {self.messages}
        USER: {question}
        """
    )
    async def _stream(self, question: str) -> openai.OpenAIDynamicConfig:
        return {
            "tools": [self.web_search, extract_content],
            "computed_fields": {
                "current_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            },
        }

    async def _step(self, question: str):
        response = await self._stream(question)
        tools_and_outputs = []
        async for chunk, tool in response:
            if tool:
                print(f"using {tool._name()} tool with args: {tool.args}")
                tools_and_outputs.append((tool, tool.call()))
            else:
                print(chunk.content, end="", flush=True)
        if response.user_message_param:
            self.messages.append(response.user_message_param)
        self.messages.append(response.message_param)
        if tools_and_outputs:
            self.messages += response.tool_message_params(tools_and_outputs)
            await self._step("")

    async def run(self):
        while True:
            question = input("(User): ")
            if question == "exit":
                break
            print(f"(User): {question}")
            print("(Assistant): ", end="", flush=True)
            await self._step(question)
            print()

This implementation includes:

1. The `_stream` method, which sets up the LLM call with the necessary tools and computed fields.
2. The `_step` method, which processes the LLM response, handles tool calls, and updates the conversation history.
3. The `run` method, which implements the main interaction loop for the agent.

The `@openai.call` and `@prompt_template` decorators are used to set up the LLM interaction and define the prompt for the agent. Note that we've explicitly specified the available tools and their usage instructions in the system prompt. We have found that this improves the performance and reliability of tool usage by the LLM.

Note how we're passing `self.web_search` as a tool, which allows the LLM to use our custom web search method that has access to the agent's state.

## 7. Running and Testing the Agent

Now that we have implemented our WebSearchAgent, let's run and test it:

In [6]:
async def main():
    web_assistant = WebSearchAgent()
    await web_assistant.run()


# Run main in a jupyter notebook
await main()

# Run main in a python script
# import asyncio
# asyncio.run(main())

(User): large language models
(Assistant): using web_search tool with args: {'queries': ['overview of large language models', 'applications of large language models', 'recent advancements in large language models', 'large language models in artificial intelligence', 'challenges in large language models']}
using extract_content tool with args: {'url': 'https://blogs.nvidia.com/blog/what-are-large-language-models-used-for/'}
using extract_content tool with args: {'url': 'https://www.techopedia.com/12-practical-large-language-model-llm-applications'}
using extract_content tool with args: {'url': 'https://www.computerworld.com/article/1627101/what-are-large-language-models-and-how-are-they-used-in-generative-ai.html'}
using extract_content tool with args: {'url': 'https://arxiv.org/abs/2307.06435'}
Large Language Models (LLMs) represent a significant advancement in artificial intelligence, particularly in natural language processing (NLP). These models are powerful deep learning algorithms

To use the WebSearchAgent, run the code above and start interacting with it by typing your questions. The agent will use web searches and content extraction to provide answers. Type 'exit' to end the interaction.

## 8. Advanced Concepts and Best Practices

When working with Tools and Agents in Mirascope, consider the following best practices:

1. **Error Handling**: Implement robust error handling in your Tools and Agent implementation.
2. **Rate Limiting**: Be mindful of rate limits when using external APIs in your Tools.
3. **Caching**: Implement caching mechanisms to improve performance and reduce API calls.
4. **Testing**: Write unit tests for your Tools and integration tests for your Agents.
5. **Modularity**: Design your Tools and Agents to be modular and reusable.
6. **Security**: Be cautious about the information you expose through your Tools and Agents.
7. **Logging**: Implement logging to track the behavior and performance of your Agents.

For more advanced usage, you can explore concepts like:

- Multi-agent systems
- Tool chaining and composition
- Dynamic tool selection
- Memory and state management for long-running agents

For more techniques and best practices in using Mirascope, refer to the following documentation:

- [Tools](https://docs.mirascope.io/latest/learn/tools/)
- [Agents](https://docs.mirascope.io/latest/learn/agents/)
- [Dynamic Configuration](https://docs.mirascope.io/latest/learn/dynamic_configuration/)
- [Chaining](https://docs.mirascope.io/latest/learn/chaining/)
- [Streams](https://docs.mirascope.io/latest/learn/streams/)

## 9. Conclusion

In this notebook, we've explored the basics of creating Tools and implementing Agents in Mirascope. We've built a WebSearchAgent that can perform web searches, extract content from webpages, and use an LLM to generate responses based on the gathered information.

This example demonstrates the power and flexibility of Mirascope in building AI applications that combine LLMs with custom tools and logic. As you continue to work with Mirascope, you'll discover more advanced features and patterns that can help you build even more sophisticated AI agents and applications.